# Adding Synthetic Hemodynamic Reponses to Data

This example notebook illustrates the functionality in `cedalion.sim.synthetic_hrf`
to create simulated datasets with added activations.

In [ ]:
# This cells setups the environment when executed in Google Colab.
try:
    import google.colab
    !curl -s https://raw.githubusercontent.com/ibs-lab/cedalion/dev/scripts/colab_setup.py -o colab_setup.py
    # Select branch with --branch "branch name" (default is "dev")
    %run colab_setup.py
except ImportError:
    pass

In [ ]:
# set this flag to True to enable interactive 3D plots
INTERACTIVE_PLOTS = False

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
import xarray as xr

import cedalion
import cedalion.dataclasses as cdc
import cedalion.datasets
import cedalion.geometry.landmarks as cd_landmarks
import cedalion.imagereco.forward_model as fw
import cedalion.models.glm as glm
import cedalion.nirs
import cedalion.plots

import cedalion.sigproc.quality as quality
import cedalion.sim.synthetic_hrf as synhrf
import cedalion.xrutils as xrutils
from cedalion import units
from cedalion.imagereco.solver import pseudo_inverse_stacked

xr.set_options(display_expand_data=False)

if INTERACTIVE_PLOTS:
    pv.set_jupyter_backend('server')
else:
    pv.set_jupyter_backend('static')

## Loading and preprocessing the dataset

This notebook uses a high-density, whole head resting state dataset recorded with a NinjaNIRS 22.

In [ ]:
rec = cedalion.datasets.get_nn22_resting_state()

geo3d = rec.geo3d
meas_list = rec._measurement_lists["amp"]

amp = rec["amp"]
amp = amp.pint.dequantify().pint.quantify("V")

display(amp)

In [ ]:
cedalion.plots.plot_montage3D(rec["amp"], geo3d)

In [ ]:
# Select channels which have at least a signal-to-noise ratio of 10
snr_thresh = 10  # the SNR (std/mean) of a channel.
snr, snr_mask = quality.snr(rec["amp"], snr_thresh)
amp_selected, masked_channels = xrutils.apply_mask(
    rec["amp"], snr_mask, "drop", "channel"
)

print(f"Removed {len(masked_channels)} channels because of low SNR.")

In [ ]:
# Calculate optical density
od = cedalion.nirs.int2od(amp_selected)

## Construct headmodel

We load the the Colin27 headmodel, since we need the geometry for image reconstruction.

In [ ]:
SEG_DATADIR, mask_files, landmarks_file = cedalion.datasets.get_icbm152_segmentation()

In [ ]:
head_ijk = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR,
    mask_files = mask_files,
    brain_surface_file= os.path.join(SEG_DATADIR, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file,
    brain_face_count=None,
    scalp_face_count=None,
    fill_holes=True,        # needs to be true, otherwise landmark calculation fails
)

# transform coordinates to a RAS coordinate system
head_ras = head_ijk.apply_transform(head_ijk.t_ijk2ras)

In [ ]:
display(head_ijk.brain)
display(head_ras.brain)

In [ ]:
head_ras.landmarks

head.landmarks contains the 4 landmarks ['Nz' 'Iz' 'LPA' 'RPA']. 
Since we want to create synthetic HRFs on the brain surface at landmark positions, we need to build the remaining 10-10 landmarks

In [ ]:
lmbuilder = cd_landmarks.LandmarksBuilder1010(head_ras.scalp, head_ras.landmarks)
all_landmarks = lmbuilder.build()
head_ras.landmarks = all_landmarks

In [ ]:
center_brain = np.mean(head_ras.brain.mesh.vertices, axis=0)

We want to build the synthetic HRFs at C3 and C4 (green dots in the image below)

In [ ]:
plt_pv = pv.Plotter()
cedalion.plots.plot_surface(plt_pv, head_ras.brain, color="#d3a6a1")
cedalion.plots.plot_surface(plt_pv, head_ras.scalp, opacity=0.1)
cedalion.plots.plot_labeled_points(
    plt_pv, head_ras.landmarks.sel(label=["C3", "C4"]), show_labels=True
)

plt_pv.camera.position = (-400, 500,400)
plt_pv.show()

## Build spatial activation pattern on brain surface for landmarks C3 and C4

The function `build_spatial_activation` is used to place a spatial activation pattern on the brain surface. The activation pattern is a Gaussian function of the geodesic distance to a seed vertex. Hence, the size of the activation is determined by the standard deviation of this Gaussian, specified by the parameter `spatial_scale`. The peak intensity in HbO is determined by the parameter `intensity_scale`. The intensity of HbR activation is specified relative to the HbO peak intensity. So if the HbO pattern describes an increase in Hbo then providing a negative factor smaller than 1 yields a decrease in HbR with smaller amplitude.
The seed vertex (integer) can be selected as the closest vertex to a given landmark:

In [ ]:
# obtain the closest vertices to C3 and C4
c3_seed = head_ras.brain.mesh.kdtree.query(head_ras.landmarks.sel(label='C3'))[1]
c4_seed = head_ras.brain.mesh.kdtree.query(head_ras.landmarks.sel(label='C4'))[1]

In [ ]:
# create the spatial activation
spatial_act = synhrf.build_spatial_activation(
    head_ras,
    c3_seed,
    spatial_scale=1 * cedalion.units.cm,
    intensity_scale=1 * units.micromolar,
    hbr_scale=-0.4,
)



The resulting `DataArray` contains an activation value for each vertex and chromophore on the brain surface.

In [ ]:
display(spatial_act)

In [ ]:
f,ax = plt.subplots(1,2,figsize=(10,5))
cedalion.plots.brain_plot(
        od,
        head_ras.landmarks,
        spatial_act.sel(chromo="HbO").pint.to("uM"),
        head_ras.brain,
        ax[0],
        camera_pos="C3",
        cmap="RdBu_r",
        vmin=-1,
        vmax=+1,
        cb_label=r"$\Delta$ HbO / µM",
        title=None,
    )
ax[0].set_title("HbO")
cedalion.plots.brain_plot(
        od,
        head_ras.landmarks,
        spatial_act.sel(chromo="HbR").pint.to("uM"),
        head_ras.brain,
        ax[1],
        camera_pos="C3",
        cmap="RdBu_r",
        vmin=-1,
        vmax=+1,
        cb_label=r"$\Delta$ HbR / µM",
        title=None,
    )
ax[1].set_title("HbR");

The following plot illustrates the effects of the `spatial_scale` and `intensity_scale` parameters:

In [ ]:
f, ax = plt.subplots(2, 3, figsize=(9, 6))
for i, spatial_scale in enumerate([0.5 * units.cm, 2 * units.cm, 3 * units.cm]):
    spatial_act = synhrf.build_spatial_activation(
        head_ras,
        c3_seed,
        spatial_scale=spatial_scale,
        intensity_scale=1 * units.micromolar,
        hbr_scale=-0.4,
    )

    cedalion.plots.brain_plot(
        od,
        head_ras.landmarks,
        spatial_act.sel(chromo="HbO").pint.to("uM"),
        head_ras.brain,
        ax[0, i],
        camera_pos="C3",
        cmap="RdBu_r",
        vmin=-1,
        vmax=+1,
        cb_label=r"$\Delta$ HbO / µM",
        title=None,
    )
    ax[0, i].set_title(f"spatial_scale: {spatial_scale.magnitude} cm")

for i, intensity_scale in enumerate(
    [
        0.5 * units.micromolar,
        1.0 * units.micromolar,
        2.0 * units.micromolar,
    ]
):
    spatial_act = synhrf.build_spatial_activation(
        head_ras,
        c3_seed,
        spatial_scale=2 * units.cm,
        intensity_scale=intensity_scale,
        hbr_scale=-0.4,
    )

    cedalion.plots.brain_plot(
        od,
        head_ras.landmarks,
        spatial_act.sel(chromo="HbO").pint.to("uM"),
        head_ras.brain,
        ax[1, i],
        camera_pos="C3",
        cmap="RdBu_r",
        vmin=-2,
        vmax=+2,
        cb_label=r"$\Delta$ HbO / µM",
        title=None,
    )
    ax[1, i].set_title(f"intensity_scale: {intensity_scale.magnitude} µM")

f.tight_layout()



For this example notebook two activations are placed below C3 and C4:

In [ ]:
int_scale_c3 = 1.0

spatial_act_c3 = synhrf.build_spatial_activation(
    head_ras,
    c3_seed,
    spatial_scale=1 * cedalion.units.cm,
    intensity_scale=int_scale_c3 * units.micromolar,
    hbr_scale=-0.4,
)
spatial_act_c4 = synhrf.build_spatial_activation(
    head_ras,
    c4_seed,
    spatial_scale=1 * cedalion.units.cm,
    intensity_scale=1 * units.micromolar,
    hbr_scale=-0.4,
)

We concatenate the two images for C3 and C4 along dimension `trial_type` to get a single `DataArray` with the spatial information for both landmarks.

In [ ]:
# concatenate the two spatial activations along a new dimension
#spatial_imgs = xr.concat(
#    [spatial_act_c3, spatial_act_c4], dim="trial_type"
#).assign_coords(trial_type=["Stim C3", "Stim C4"])

spatial_imgs = xr.concat(
    [spatial_act_c3], dim="trial_type"
).assign_coords(trial_type=["Stim C3"])

spatial_imgs

## Plots of spatial patterns

Using the helper function `cedalion.plots.brain_plot`, the created activations
on the brain surface below C3 and C4 are plotted:

In [ ]:
f, ax = plt.subplots(1,1, figsize=(10,10))

cedalion.plots.brain_plot(
    od,
    head_ras.landmarks,
    spatial_imgs.sel(trial_type="Stim C3", chromo="HbO").pint.to("uM"),
    head_ras.brain,
    ax,
    camera_pos="C3",
    cmap="RdBu_r",
    vmin=-1,
    vmax=+1,
    cb_label=r"$\Delta$ HbO / µM",
)

f.tight_layout()

f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_raw_spatial_map_{int_scale_c3}_hbo.png", dpi=300)
f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_raw_spatial_map_{int_scale_c3}_hbo.svg", dpi=300)
f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_raw_spatial_map_{int_scale_c3}_hbo.pdf", dpi=300)

f.show()

f, ax = plt.subplots(1,1, figsize=(10,10))

cedalion.plots.brain_plot(
    od,
    head_ras.landmarks,
    spatial_imgs.sel(trial_type="Stim C3", chromo="HbR").pint.to("uM"),
    head_ras.brain,
    ax,
    camera_pos="C3",
    cmap="RdBu_r",
    vmin=-1,
    vmax=+1,
    cb_label=r"$\Delta$ HbO / µM",
)

f.tight_layout()

f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_raw_spatial_map_{int_scale_c3}_hbr.png", dpi=300)
f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_raw_spatial_map_{int_scale_c3}_hbr.svg", dpi=300)
f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_raw_spatial_map_{int_scale_c3}_hbr.pdf", dpi=300)

f.show()

## Image Reconstruction

We load the precomputed Adot matrix to be able to map from image to channel space. (For details see image_reconstruction example notebook).

In [ ]:
Adot = cedalion.datasets.get_precomputed_sensitivity("nn22_resting", "icbm152")

In [ ]:
# we only consider brain vertices, not scalp
Adot_brain = Adot[:, (Adot.is_brain).values,:]

# drop the pruned channels
Adot_brain = Adot_brain.sel(channel=od.channel)
Adot_brain

The forward model and image reconstruction translate between timeseries of different wavelengths in channel space and time series of different chromophores in image space. To this end the image reconstruction operates on stacked arrays in which the dimensions 'channel' and 'wavelength' are stacked to form a new dimension 'flat_channel'. Likewise the dimensions 'vertex' and 'chromo' are stacked as 'flat_vertex'.

In [ ]:
Adot_stacked = fw.ForwardModel.compute_stacked_sensitivity(Adot_brain)

Adot_stacked = Adot_stacked.pint.quantify()
Adot_stacked


invert the sensitivity matrix:

In [ ]:
Adot_inverted = pseudo_inverse_stacked(Adot_stacked)
Adot_inverted = Adot_inverted.pint.quantify()
Adot_inverted

To multiply the spatial image with the sensitivity matrix the spatial image's vertex
and chromo dimensions must be stacked, too.

In [ ]:
spatial_imgs_stacked = fw.stack_flat_vertex(spatial_imgs)
display(spatial_imgs_stacked)

We can now map our spatial patterns to channel space:

In [ ]:
spatial_chan_stacked = Adot_stacked @ spatial_imgs_stacked
spatial_chan_stacked

In [ ]:
spatial_chan = fw.unstack_flat_channel(spatial_chan_stacked)

In [ ]:
Adot_inverted

In [ ]:
spatial_img_reco = Adot_inverted @ fw.stack_flat_channel(spatial_chan).sel(trial_type="Stim C3")

Show the spatial activation in channel space with a scalp plot.

In [ ]:
fig, ax = plt.subplots(1, 1)
# adjust plot size
fig.set_size_inches(10, 10)
cedalion.plots.scalp_plot(
    od,
    rec.geo3d,
    spatial_chan.sel(trial_type="Stim C3", wavelength=850),
    ax,
    cmap="YlOrRd",
    #title="850nm, activation under C3",
    #vmin=spatial_chan.values.min(),
    #vmax=spatial_chan.values.max(),
    cb_label="Max Peak Amplitude",
)
plt.tight_layout()
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_chan_{int_scale_c3}_850nm.png", dpi=300)
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_chan_{int_scale_c3}_850nm.svg", dpi=300)
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/syn_hrf_chan_{int_scale_c3}_850nm.pdf", dpi=300)
plt.show()

Get top 5 channels for each trial type where synthetic activation is highest

In [ ]:
roi_chans_c3 = spatial_chan.channel[
    spatial_chan.sel(trial_type="Stim C3").max("wavelength").argsort()[-1:].values
].values
roi_chans_c3

## Concentration Scale

The activations were simulataed in image space with a peak concentration change of 1 µM 
in one vertex. The change in optical density in one channel reflects concentration
changes in the ensemble of vertices that this channel is sensitive to.

When applying the Beer-Lambert-transformation in channel space, a change in 
concentration is calculated for each channel. However, the scales of these concentration
changes and the concentration changes in single vertices are not the same.

Here, a correction factor is calculated to scale the activation in channel
space to 1uM.


In [ ]:
dpf = xr.DataArray(
    [6, 6],
    dims="wavelength",
    coords={"wavelength": rec["amp"].wavelength},
)

## HRFs in channel space

So far the notebook focused on the spatial extent of the activation. 
To build the temporal HRF model we use the same functionality that generates hrf regressors for the GLM.

First we select a basis function, which defines the temporal shape of the HRF.

In [ ]:
basis_fct = glm.Gamma(tau=0 * units.s, sigma=3 * units.s, T=3 * units.s)

In [ ]:
od.time

A Stim DataFrame, which contains the onset, duration and amplitude of the synthetic HRFs, is created.

In [ ]:
#stim_df = synhrf.build_stim_df(
#    max_time=od.time.values[-1] * units.seconds,
#    trial_types=["Stim C3",],
#    min_interval=12 * units.seconds,
#    max_interval=16 * units.seconds,
#    min_stim_dur = 10 * units.seconds,
#    max_stim_dur = 10 * units.seconds,
#    min_stim_value = 1.0,
#    max_stim_value = 1.0,
#    order="random",
#)

import pandas as pd

stim_df = pd.DataFrame([
    {"onset": 12.26, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 36.26, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 60.64, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 85.70, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 111.60, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 136.27, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 160.13, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 184.50, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 202.85, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 228.37, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 251.06, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 275.27, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 301.18, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"},
    {"onset": 325.39, "duration": 10.0, "value": 1.0, "trial_type": "Stim C3"}
])


In [ ]:
stim_df

In [ ]:
len(stim_df)

# Select intensity scale

In [ ]:
# add time axis with one time point so we can convert to conc
spatial_chan_w_time = spatial_chan.expand_dims("time")
spatial_chan_w_time = spatial_chan_w_time.assign_coords(time=[0])
spatial_chan_w_time.time.attrs["units"] = "second"
display(spatial_chan_w_time)
spatial_chan_conc = cedalion.nirs.od2conc(
    spatial_chan_w_time, geo3d, dpf, spectrum="prahl"
)
# rescale so that synthetic hrfs add 1 micromolar at peak.
int_scaling_factor = 0.6
intensity_scale = int_scaling_factor * units.micromolar
rescale_factor = (intensity_scale / spatial_chan_conc.max())
display(rescale_factor)
spatial_chan *= rescale_factor

We can now use our stim dataframe, basis function, and spatial information to create the synthetic HRF timeseries

In [ ]:
syn_ts = synhrf.build_synthetic_hrf_timeseries(od, stim_df, basis_fct, spatial_chan)

We get a synthetic HRF timeseries for each channel, trial_type and chromo / wavelength

In [ ]:
syn_ts

Sum the synthetic timeseries over trial_type dimension, so it has the same shape as the resting state data

In [ ]:
syn_ts_sum = syn_ts.sum(dim='trial_type')
syn_ts_sum

## Adding HRFs to measured data

Here, the simulated activations are combined with physiological noise by adding
the synthetic HRFs to the resting state dataset:


In [ ]:
od_w_hrf = od + syn_ts_sum

## Recover the HRFs again

In the following, the added activations should be extracted from the simulated dataset
again. To this end, the data is frequency filtered and block averages are calculated.

In [ ]:
od_w_hrf_filtered = od_w_hrf.cd.freq_filter(fmin=0.02, fmax=0.5, butter_order=4)

In [ ]:
od_w_hrf_filtered

In [ ]:
conc_w_hrf_filtered = cedalion.nirs.od2conc(
    od_w_hrf_filtered, geo3d, dpf, spectrum="prahl"
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Epoch extraction & baseline correction ---

epochs = od_w_hrf_filtered.cd.to_epochs(
    stim_df, ["Stim C3"],
    before=5 * units.seconds,
    after=20 * units.seconds,
)

baseline = epochs.sel(reltime=(epochs.reltime < 0)).mean("reltime")
epochs_blcorrected = epochs - baseline

blockavg_od = epochs.mean("epoch")

epochs_conc = conc_w_hrf_filtered.cd.to_epochs(
    stim_df, ["Stim C3"],
    before=5 * units.seconds,
    after=20 * units.seconds,
)

baseline_conc = epochs_conc.sel(reltime=(epochs_conc.reltime < 0)).mean("reltime")
epochs_blcorrected_conc = epochs_conc - baseline_conc

# --- compute blockaverage + STD ---
blockavg = epochs_blcorrected_conc.mean("epoch")
blockstd = epochs_blcorrected_conc.std("epoch")

# meta
n_roi = roi_chans_c3.size
time = blockavg.reltime.values

# colors HbO/HbR (no transparent lines)
col_hbo = "red"
col_hbr = "blue"


In [ ]:
fig, axes = plt.subplots(1, n_roi, figsize=(10*n_roi, 6), sharey=True)

if n_roi == 1:
    axes = [axes]

for ax, ch in zip(axes, roi_chans_c3):

    # HbO
    for ep in epochs_blcorrected_conc.sel(chromo="HbO", channel=ch):
        ax.plot(time, ep, color=col_hbo, linewidth=0.7)

    # HbR
    for ep in epochs_blcorrected_conc.sel(chromo="HbR", channel=ch):
        ax.plot(time, ep, color=col_hbr, linewidth=0.7)

    ax.grid(True)

axes[0].set_ylabel("Δ Concentration (µM)", fontsize=20)
axes[0].set_xlabel("Time (Seconds)", fontsize=20)
plt.ylim(-1.1, 1.6)
plt.tight_layout()

plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_all_epochs_{int_scaling_factor}.png", dpi=300)
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_all_epochs_{int_scaling_factor}.svg", dpi=300)
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_all_epochs_{int_scaling_factor}.pdf", dpi=300)

plt.show()

In [ ]:
fig, axes = plt.subplots(1, n_roi, figsize=(10*n_roi, 6), sharey=True)

if n_roi == 1:
    axes = [axes]

for ax, ch in zip(axes, roi_chans_c3):

    # HbO background STD band
    ax.fill_between(
        time,
        blockavg.sel(chromo="HbO", channel=ch) - blockstd.sel(chromo="HbO", channel=ch),
        blockavg.sel(chromo="HbO", channel=ch) + blockstd.sel(chromo="HbO", channel=ch),
        color=col_hbo,
        alpha=0.15  # „Hintergrund“, nicht deckende Linie
    )

    # HbR background STD band
    ax.fill_between(
        time,
        blockavg.sel(chromo="HbR", channel=ch) - blockstd.sel(chromo="HbR", channel=ch),
        blockavg.sel(chromo="HbR", channel=ch) + blockstd.sel(chromo="HbR", channel=ch),
        color=col_hbr,
        alpha=0.15
    )

    # blockaverage lines
    ax.plot(time, blockavg.sel(chromo="HbO", channel=ch), color=col_hbo, linewidth=2)
    ax.plot(time, blockavg.sel(chromo="HbR", channel=ch), color=col_hbr, linewidth=2)

    ax.set_title(f"Channel {ch}", fontsize=22)
    ax.grid(True)
    ax.tick_params(axis='y', labelsize=16)
    ax.tick_params(axis='x', labelsize=16)

axes[0].set_ylabel("Δ Concentration (µM)", fontsize=20)
axes[0].set_xlabel("Time (Seconds)", fontsize=20)
plt.ylim(-0.5, 1.1)
plt.tight_layout()
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_blockaverage_std_{int_scaling_factor}.png", dpi=300)
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_blockaverage_std_{int_scaling_factor}.svg", dpi=300)
plt.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_blockaverage_std_{int_scaling_factor}.pdf", dpi=300)

plt.show()


## Map block average back to brain surface

We map our extracted block averages back to the brain surface to visualize the recovered HRFs activation for Stim C3.
We can compare it to the synthetic HRF image we created earlier.

In [ ]:
blockaverage_img = Adot_inverted @ fw.stack_flat_channel(blockavg_od)

In [ ]:
# build an xindex to use .sel along the chromo dimension
blockaverage_img = blockaverage_img.set_xindex("chromo")

In [ ]:
blockaverage_img_mean = blockaverage_img.sel(reltime=slice(5,15)).mean("reltime")

In [ ]:
blockaverage_img_mean.sel(chromo="HbO").pint.to("uM").max() / rescale_factor * int_scaling_factor

In [ ]:
f, ax = plt.subplots(1,1, figsize=(10,10))

cedalion.plots.brain_plot(
    od,
    head_ras.landmarks,
    blockaverage_img_mean.sel(chromo="HbO").pint.to("uM"),
    head_ras.brain,
    ax,
    camera_pos="C3",
    cmap="RdBu_r",
    vmin=-2,
    vmax=2,
    cb_label=r"$\Delta$ HbO / µM",
    title="C3 Activation",
)
# save plot as png, svg, and pdf
f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_blockaverage_img_{int_scaling_factor}.png", dpi=300)
f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_blockaverage_img_{int_scaling_factor}.svg", dpi=300)
f.savefig(f"/home/thomas/Dokumente/Master/Master_Thesis/HD-DOT Classification/writing/images/syn_hrf/validation/syn_hrf_blockaverage_img_{int_scaling_factor}.pdf", dpi=300)
plt.show()